<a href="https://colab.research.google.com/github/vitthal-bhandari/IMDb-Sentiment-Analysis/blob/master/IMDb_Movies_Review_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [2]:
", ".join(tfds.list_builders())

'abstract_reasoning, accentdb, aeslc, aflw2k3d, ag_news_subset, ai2_arc, ai2_arc_with_ir, amazon_us_reviews, anli, arc, bair_robot_pushing_small, bccd, beans, big_patent, bigearthnet, billsum, binarized_mnist, binary_alpha_digits, blimp, bool_q, c4, caltech101, caltech_birds2010, caltech_birds2011, cars196, cassava, cats_vs_dogs, celeb_a, celeb_a_hq, cfq, chexpert, cifar10, cifar100, cifar10_1, cifar10_corrupted, citrus_leaves, cityscapes, civil_comments, clevr, clic, clinc_oos, cmaterdb, cnn_dailymail, coco, coco_captions, coil100, colorectal_histology, colorectal_histology_large, common_voice, coqa, cos_e, cosmos_qa, covid19sum, crema_d, curated_breast_imaging_ddsm, cycle_gan, deep_weeds, definite_pronoun_resolution, dementiabank, diabetic_retinopathy_detection, div2k, dmlab, downsampled_imagenet, dsprites, dtd, duke_ultrasound, emnist, eraser_multi_rc, esnli, eurosat, fashion_mnist, flic, flores, food101, forest_fires, fuss, gap, geirhos_conflict_stimuli, genomics_ood, german_credit

In [3]:
#loading the moview review test and train splits
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train", with_info=True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews", split="test", as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYN8D5E/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYN8D5E/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
#reading the associated info
ds_info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [5]:
#printing a data sample
for example, label in imdb_train.take(1):
  print(example, '\n', label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


In [7]:
tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set=set()
MAX_TOKENS=0
for example, label in imdb_train:
  some_tokens=tokenizer.tokenize(example.numpy())
  if MAX_TOKENS<len(some_tokens):
    MAX_TOKENS=len(some_tokens)
  vocabulary_set.update(some_tokens)

In [9]:
#encode the tokens
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set,tokenizer=tokenizer)

In [10]:
vocab_size = imdb_encoder.vocab_size
print(vocab_size, MAX_TOKENS)

93931 2525


In [11]:
for example, label in imdb_train.take(1):
  print(example)
  encoded = imdb_encoder.encode(example.numpy())
  print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This 

In [12]:
from tensorflow.keras.preprocessing import sequence
def encode_pad_transform(sample):
  encoded = imdb_encoder.encode(sample.numpy())
  pad = sequence.pad_sequences([encoded], padding='post', maxlen=150)
  return np.array(pad[0], dtype=np.int64)

def encode_tf_fn(sample, label):
  encoded = tf.py_function(encode_pad_transform, inp=[sample], Tout=(tf.int64))
  encoded.set_shape([None])
  label.set_shape([])
  return encoded, label

In [14]:
#transforming a subset of the dataset
subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)
for review, label in tst.take(1):
  print(review, label)
  print(imdb_encoder.decode(review))

tf.Tensor(
[62324 81331 27732 45835 49888  5671 88629 81668 31374 61545 92737 44234
 27457 59410 29675 43765 52869 39138 38155 19367 17769 78437 43903 92963
 91734 31374 87543 53817 72738 92737 70328 21313 87543 19367 37424  4370
 46026 39032 43903  5671 12508 68703 36666 62324  5671 16095 27732 73973
 69832   327 82324 53589 26155 20965 30644 38095 79012 60016 41450 13572
 89357 57574 79012 71689 87543 13168 24269 64044 78796 65966 13072 31743
  8582 25509 44161 91386  9066 22024 16367 59410 81331 85299 78437 60331
 30644  3157 37069 92737 60331  5671  3467 81331 52129 63582 71336 60779
 70275 67152 78418 87088  3467 74741 38155 58235 39909 43903 63225  1092
 12508 39909 27457 59410 12508 87500 60010 67152  4370 20209 51408 70423
 76567     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

In [15]:
#transforming the entire data
encoded_train = imdb_train.map(encode_tf_fn)
encoded_test = imdb_test.map(encode_tf_fn)

In [17]:
#building a simple LSTM model
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model=tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True, batch_input_shape=[batch_size, None]),
                             tf.keras.layers.LSTM(rnn_units),
                             tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [18]:
#configuring parameters
vocab_size = imdb_encoder.vocab_size

embedding_dim=64

rnn_units=64

BATCH_SIZE=100

In [19]:
#final model
model = build_model_lstm(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (100, None, 64)           6011584   
                                                                 
 lstm (LSTM)                 (100, 64)                 33024     
                                                                 
 dense (Dense)               (100, 1)                  65        
                                                                 
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [20]:
#training the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
encoded_train_batched = encoded_train.batch(BATCH_SIZE)
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 46s 154ms/step - loss: 0.4141 - accuracy: 0.8010 - precision: 0.7859 - recall: 0.8276
Epoch 2/10
250/250 [==============================] - 38s 153ms/step - loss: 0.1758 - accuracy: 0.9350 - precision: 0.9348 - recall: 0.9354
Epoch 3/10
250/250 [==============================] - 37s 149ms/step - loss: 0.1172 - accuracy: 0.9596 - precision: 0.9603 - recall: 0.9588
Epoch 4/10
250/250 [==============================] - 37s 149ms/step - loss: 0.0798 - accuracy: 0.9739 - precision: 0.9751 - recall: 0.9726
Epoch 5/10
250/250 [==============================] - 38s 151ms/step - loss: 0.0495 - accuracy: 0.9829 - precision: 0.9829 - recall: 0.9829
Epoch 6/10
250/250 [==============================] - 38s 152ms/step - loss: 0.0304 - accuracy: 0.9897 - precision: 0.9901 - recall: 0.9893
Epoch 7/10
250/250 [==============================] - 39s 154ms/step - loss: 0.0443 - accuracy: 0.9853 - precision: 0.9864 - recall: 0.9842
Epoch 8/10
250/250 [

In [21]:
#evaluate on test set
model.evaluate(encoded_test.batch(BATCH_SIZE))

250/250 [==============================] - 34s 129ms/step - loss: 0.9931 - accuracy: 0.7969 - precision: 0.7309 - recall: 0.9398


[0.9931180477142334, 0.7968800067901611, 0.730869710445404, 0.9398400187492371]

Train accuracy is 99.66%
but Test accuracy is 79.69%.
So model is probably overfitting.
Hence we will add a dropout layer and train again with regularization.

In [27]:
#building a simple LSTM model with dropout
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size, drouput_frac):
  model=tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True, batch_input_shape=[batch_size, None]),
                             tf.keras.layers.LSTM(rnn_units),
                             tf.keras.layers.Dropout(drouput_frac),
                             tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model


In [23]:
#final model
model = build_model_lstm(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE,
    drouput_frac=0.3
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (100, None, 64)           6011584   
                                                                 
 lstm_1 (LSTM)               (100, 64)                 33024     
                                                                 
 dropout (Dropout)           (100, 64)                 0         
                                                                 
 dense_1 (Dense)             (100, 1)                  65        
                                                                 
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [24]:
#training the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
encoded_train_batched = encoded_train.batch(BATCH_SIZE)
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 42s 152ms/step - loss: 0.4178 - accuracy: 0.8015 - precision: 0.7901 - recall: 0.8210
Epoch 2/10
250/250 [==============================] - 37s 150ms/step - loss: 0.1789 - accuracy: 0.9360 - precision: 0.9339 - recall: 0.9384
Epoch 3/10
250/250 [==============================] - 38s 152ms/step - loss: 0.1098 - accuracy: 0.9632 - precision: 0.9615 - recall: 0.9651
Epoch 4/10
250/250 [==============================] - 37s 148ms/step - loss: 0.0646 - accuracy: 0.9795 - precision: 0.9801 - recall: 0.9790
Epoch 5/10
250/250 [==============================] - 37s 149ms/step - loss: 0.0455 - accuracy: 0.9858 - precision: 0.9872 - recall: 0.9843
Epoch 6/10
250/250 [==============================] - 37s 148ms/step - loss: 0.0505 - accuracy: 0.9837 - precision: 0.9837 - recall: 0.9838
Epoch 7/10
250/250 [==============================] - 38s 150ms/step - loss: 0.0456 - accuracy: 0.9854 - precision: 0.9867 - recall: 0.9840
Epoch 8/10
250/250 [

In [25]:
model.evaluate(encoded_test.batch(BATCH_SIZE))

250/250 [==============================] - 33s 124ms/step - loss: 0.8565 - accuracy: 0.8342 - precision: 0.8175 - recall: 0.8605


[0.8565104007720947, 0.8342000246047974, 0.8175116181373596, 0.860480010509491]

Train accuracy is 99.58%
but Test accuracy is 83.42%.
So regularization has helped.
We'll try to increase dropout value and compare the result.

In [28]:
#final model
model = build_model_lstm(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE,
    drouput_frac=0.5
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (100, None, 64)           6011584   
                                                                 
 lstm_2 (LSTM)               (100, 64)                 33024     
                                                                 
 dropout_1 (Dropout)         (100, 64)                 0         
                                                                 
 dense_2 (Dense)             (100, 1)                  65        
                                                                 
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [29]:
#training the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'Precision', 'Recall'])
encoded_train_batched = encoded_train.batch(BATCH_SIZE)
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 42s 150ms/step - loss: 0.4455 - accuracy: 0.7799 - precision: 0.7664 - recall: 0.8051
Epoch 2/10
250/250 [==============================] - 37s 149ms/step - loss: 0.1839 - accuracy: 0.9320 - precision: 0.9312 - recall: 0.9330
Epoch 3/10
250/250 [==============================] - 37s 148ms/step - loss: 0.1049 - accuracy: 0.9651 - precision: 0.9645 - recall: 0.9657
Epoch 4/10
250/250 [==============================] - 37s 150ms/step - loss: 0.0895 - accuracy: 0.9712 - precision: 0.9693 - recall: 0.9731
Epoch 5/10
250/250 [==============================] - 37s 148ms/step - loss: 0.0548 - accuracy: 0.9822 - precision: 0.9821 - recall: 0.9824
Epoch 6/10
250/250 [==============================] - 37s 148ms/step - loss: 0.0455 - accuracy: 0.9860 - precision: 0.9876 - recall: 0.9843
Epoch 7/10
250/250 [==============================] - 37s 148ms/step - loss: 0.0363 - accuracy: 0.9890 - precision: 0.9897 - recall: 0.9883
Epoch 8/10
250/250 [

In [30]:
model.evaluate(encoded_test.batch(BATCH_SIZE))

250/250 [==============================] - 32s 121ms/step - loss: 0.6421 - accuracy: 0.8333 - precision: 0.8423 - recall: 0.8202


[0.6420857906341553,
 0.8333200216293335,
 0.8423301577568054,
 0.8201599717140198]

Train accuracy is 98.96%
but Test accuracy is 83.33%.
So increasing the dropout fraction is not beneficial. We'll move on to a more complex architecture made up of BiLSTM's.

**Bi-LSTM**

In [43]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=50

In [41]:
#new model with bi-lstm
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size, dropout):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True, batch_input_shape=[batch_size, None]),
                               tf.keras.layers.Dropout(dropout),
                               tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True, dropout=0.25)),
                               tf.keras.layers.Dropout(dropout),
                               tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, dropout=0.25)),
                               tf.keras.layers.Dropout(dropout),
                               tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [44]:
bilstm=build_model_bilstm(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE,
    dropout=0.2
)
bilstm.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (50, None, 128)           12023168  
                                                                 
 dropout_5 (Dropout)         (50, None, 128)           0         
                                                                 
 bidirectional_4 (Bidirectio  (50, None, 128)          98816     
 nal)                                                            
                                                                 
 dropout_6 (Dropout)         (50, None, 128)           0         
                                                                 
 bidirectional_5 (Bidirectio  (50, 128)                98816     
 nal)                                                            
                                                                 
 dropout_7 (Dropout)         (50, 128)                

In [45]:
bilstm.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

In [46]:
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [47]:
bilstm.fit(encoded_train_batched, epochs=5)

Epoch 1/5
500/500 [==============================] - 111s 192ms/step - loss: 0.3794 - accuracy: 0.8240 - precision: 0.8168 - recall: 0.8354
Epoch 2/5
500/500 [==============================] - 98s 197ms/step - loss: 0.1434 - accuracy: 0.9485 - precision: 0.9473 - recall: 0.9498
Epoch 3/5
500/500 [==============================] - 98s 195ms/step - loss: 0.0676 - accuracy: 0.9776 - precision: 0.9787 - recall: 0.9764
Epoch 4/5
500/500 [==============================] - 98s 196ms/step - loss: 0.0498 - accuracy: 0.9828 - precision: 0.9837 - recall: 0.9819
Epoch 5/5
500/500 [==============================] - 98s 196ms/step - loss: 0.0463 - accuracy: 0.9840 - precision: 0.9842 - recall: 0.9838


In [48]:
bilstm.evaluate(encoded_test.batch(BATCH_SIZE))

500/500 [==============================] - 56s 102ms/step - loss: 0.7235 - accuracy: 0.8388 - precision: 0.8082 - recall: 0.8886


[0.7235361933708191,
 0.8388400077819824,
 0.8081932663917542,
 0.8885599970817566]

Train accuracy = 98.40% and Test Accuracy = 83.89%